In [1]:
# import the usual
# import matplotlib.pyplot as plt
import pandas as pd
# import seaborn as sns
import numpy as np
# import hashlib 
import datetime
#import surprise
#from surprise import SVD
%matplotlib inline
pd.set_option('display.max_columns', 500)

from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics

In [2]:
data_profile = pd.read_csv(r"F:\Data_Repository\lastfm\usersha1-profile.tsv", sep="\t", error_bad_lines=False, header = None, names = ['userhashid','gender','age','country','signup_date'])

In [4]:
data_profile.head()

,userhashid,gender,age,country,signup_date
0,00000c289a1829a808ac09c00daf10bc3c4e223b,f,22.0,Germany,"Feb 1, 2007"
1,00001411dc427966b17297bf4d69e7e193135d89,f,NaN,Canada,"Dec 4, 2007"
2,00004d2ac9316e22dc007ab2243d6fcb239e707d,NaN,NaN,Germany,"Sep 1, 2006"
3,000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,m,19.0,Mexico,"Apr 28, 2008"
4,00007a47085b9aab8af55f52ec8846ac479ac4fe,m,28.0,United States,"Jan 27, 2006"


In [5]:
# data_artist = pd.read_csv(r"F:\Data_Repository\lastfm\usersha1-artmbid-artname-plays.tsv", sep="\t", error_bad_lines=False, header = None,
#                           names = ['userhashid','artistid','artistName','plays'])

In [6]:
# data_artist

In [7]:
# data_artist['userhashid'].nunique()

In [8]:
%%time
#Music Data Loading
df = pd.read_csv(r"F:\Data_Repository\lastfm\userid-timestamp-artid-artname-traid-traname.tsv", sep="\t", error_bad_lines=False, header = None)

b'Skipping line 2120260: expected 6 fields, saw 8\n'
b'Skipping line 2446318: expected 6 fields, saw 8\n'
b'Skipping line 11141081: expected 6 fields, saw 8\n'
b'Skipping line 11152099: expected 6 fields, saw 12\nSkipping line 11152402: expected 6 fields, saw 8\n'
b'Skipping line 11882087: expected 6 fields, saw 8\n'
b'Skipping line 12902539: expected 6 fields, saw 8\nSkipping line 12935044: expected 6 fields, saw 8\n'
b'Skipping line 17589539: expected 6 fields, saw 8\n'


In [9]:
df.columns = ['userid', 'timestamp', 'artistid', 'artist', '1', 'song']

In [10]:
df.head()

,userid,timestamp,artistid,artist,1,song
0,user_000001,2009-05-04T23:08:57Z,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007
1,user_000001,2009-05-04T13:54:10Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15)
2,user_000001,2009-05-04T13:52:04Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15)
3,user_000001,2009-05-04T13:42:52Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15)
4,user_000001,2009-05-04T13:42:11Z,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15)


In [11]:
df['time'] = pd.to_datetime(df['timestamp'])
df.drop(['timestamp'], axis=1, inplace=True)

In [12]:
df.head()

,userid,artistid,artist,1,song,time
0,user_000001,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Deep Dish,NaN,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007,2009-05-04 23:08:57
1,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Composition 0919 (Live_2009_4_15),2009-05-04 13:54:10
2,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc2 (Live_2009_4_15),2009-05-04 13:52:04
3,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Hibari (Live_2009_4_15),2009-05-04 13:42:52
4,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,坂本龍一,NaN,Mc1 (Live_2009_4_15),2009-05-04 13:42:11


### Removing artist name and song id (possibly) from the user_songs_df dataframe

In [13]:
user_songs_df = df.drop(['artist'], axis=1, inplace=False)
user_songs_df.columns = ['userid', 'artistid', 'songid', 'song', 'time']
user_songs_df = user_songs_df.drop(['songid'], axis=1, inplace=False)

In [14]:
user_songs_df.head()

,userid,artistid,song,time
0,user_000001,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007,2009-05-04 23:08:57
1,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,Composition 0919 (Live_2009_4_15),2009-05-04 13:54:10
2,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,Mc2 (Live_2009_4_15),2009-05-04 13:52:04
3,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,Hibari (Live_2009_4_15),2009-05-04 13:42:52
4,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,Mc1 (Live_2009_4_15),2009-05-04 13:42:11


In [15]:
print('Number of songs : ' + str(user_songs_df['song'].nunique()))

Number of songs : 1083471


In [16]:
user_unique = user_songs_df['userid'].drop_duplicates()

In [17]:
type(user_unique)

pandas.core.series.Series

In [18]:
#user_unique = map(lambda s: s.strip(), user_unique)

In [19]:
user_unique = user_unique.reset_index()
user_unique = user_unique.drop('index', axis = 1)

In [20]:
user_unique

,userid
0,user_000001
1,user_000002
2,user_000003
3,user_000004
4,user_000005
5,user_000006
6,user_000007
7,user_000008
8,user_000009
9,user_000010


In [21]:
# user_unique2 = user_songs_df.copy()

In [22]:
# user_unique2['username'] = user_songs_df['userid'].drop_duplicates()

In [23]:
# user_unique2.head()

In [24]:
data_profile['username'] = user_unique

In [25]:
data_profile[:992]

,userhashid,gender,age,country,signup_date,username
0,00000c289a1829a808ac09c00daf10bc3c4e223b,f,22.0,Germany,"Feb 1, 2007",user_000001
1,00001411dc427966b17297bf4d69e7e193135d89,f,NaN,Canada,"Dec 4, 2007",user_000002
2,00004d2ac9316e22dc007ab2243d6fcb239e707d,NaN,NaN,Germany,"Sep 1, 2006",user_000003
3,000063d3fe1cf2ba248b9e3c3f0334845a27a6bf,m,19.0,Mexico,"Apr 28, 2008",user_000004
4,00007a47085b9aab8af55f52ec8846ac479ac4fe,m,28.0,United States,"Jan 27, 2006",user_000005
5,0000c176103e538d5c9828e695fed4f7ae42dd01,m,20.0,United Kingdom,"Jan 14, 2006",user_000006
6,0000ee7dd906373efa37f4e1185bfe1e3f8695ae,m,17.0,Finland,"Nov 17, 2007",user_000007
7,0000ef373bbd0d89ce796abae961f2705e8c1faf,f,22.0,Poland,"May 23, 2007",user_000008
8,0000f687d4fe9c1ed49620fbc5ed5b0d7798ea20,f,24.0,Spain,"Nov 4, 2008",user_000009
9,0001399387da41d557219578fb08b12afa25ab67,m,NaN,Ukraine,"Aug 17, 2008",user_000010


In [26]:
data_profile['userhashid'].nunique()

359347

In [27]:
# create user profile data after merging
#user_songs_df[pd.Series( user_songs_df['userid'].unique() ) ]
#user_songs_df['userid'].value_counts().sort_index()
#user_songs_df [ pd.Series ( user_songs_df['userid'].unique().tolist() ) ]

In [28]:
user_songs_df.head()

,userid,artistid,song,time
0,user_000001,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007,2009-05-04 23:08:57
1,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,Composition 0919 (Live_2009_4_15),2009-05-04 13:54:10
2,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,Mc2 (Live_2009_4_15),2009-05-04 13:52:04
3,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,Hibari (Live_2009_4_15),2009-05-04 13:42:52
4,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,Mc1 (Live_2009_4_15),2009-05-04 13:42:11


In [29]:
user_songs_df = user_songs_df.merge(data_profile[:992], left_on='userid', right_on='username', how = 'outer')

In [30]:
user_songs_df.head()

,userid,artistid,song,time,userhashid,gender,age,country,signup_date,username
0,user_000001,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007,2009-05-04 23:08:57,00000c289a1829a808ac09c00daf10bc3c4e223b,f,22.0,Germany,"Feb 1, 2007",user_000001
1,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,Composition 0919 (Live_2009_4_15),2009-05-04 13:54:10,00000c289a1829a808ac09c00daf10bc3c4e223b,f,22.0,Germany,"Feb 1, 2007",user_000001
2,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,Mc2 (Live_2009_4_15),2009-05-04 13:52:04,00000c289a1829a808ac09c00daf10bc3c4e223b,f,22.0,Germany,"Feb 1, 2007",user_000001
3,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,Hibari (Live_2009_4_15),2009-05-04 13:42:52,00000c289a1829a808ac09c00daf10bc3c4e223b,f,22.0,Germany,"Feb 1, 2007",user_000001
4,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,Mc1 (Live_2009_4_15),2009-05-04 13:42:11,00000c289a1829a808ac09c00daf10bc3c4e223b,f,22.0,Germany,"Feb 1, 2007",user_000001


In [31]:
user_songs_df = user_songs_df.drop(['userhashid', 'signup_date', 'username'], axis = 1)

In [32]:
#user_songs_df = user_songs_df.rename ( columns = {"userid_x" : "userid"} )

In [33]:
user_songs_df.head()

,userid,artistid,song,time,gender,age,country
0,user_000001,f1b1cf71-bd35-4e99-8624-24a6e15f133a,Fuck Me Im Famous (Pacha Ibiza)-09-28-2007,2009-05-04 23:08:57,f,22.0,Germany
1,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,Composition 0919 (Live_2009_4_15),2009-05-04 13:54:10,f,22.0,Germany
2,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,Mc2 (Live_2009_4_15),2009-05-04 13:52:04,f,22.0,Germany
3,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,Hibari (Live_2009_4_15),2009-05-04 13:42:52,f,22.0,Germany
4,user_000001,a7f7df4a-77d8-4f12-8acd-5c60c93f4de8,Mc1 (Live_2009_4_15),2009-05-04 13:42:11,f,22.0,Germany


In [34]:
def get_unique_count(column):
    return len(set(column))

In [35]:
len(user_songs_df)

19098853

In [36]:
# df_songs_heard_10_users = user_songs_df.groupby("song").filter(lambda x: get_unique_count( x['userid']) > 10)

In [37]:
# len(df_songs_heard_10_users)

In [38]:
# df_songs_heard_10_users.head()

In [39]:
# print('Number of songs heard by at least 10 users : ' + str(df_songs_heard_10_users['song'].nunique()))
# print('Number of users in this dataframe : ' + str(df_songs_heard_10_users['userid'].nunique()))

In [40]:
# %%time
# df_songs_heard_100_users = user_songs_df.groupby("song").filter(lambda x: get_unique_count(x['userid'])>100)

In [41]:
# len(df_songs_heard_100_users)

In [42]:
# df_songs_heard_100_users.head()

In [43]:
# print('Number of songs heard by at least 100 users : ' + str(df_songs_heard_100_users['song'].nunique()))
# print('Number of users in this dataframe : ' + str(df_songs_heard_100_users['userid'].nunique()))

### Creation of timeslots

In [44]:
# df_songs_heard_100_users['hour'] = df_songs_heard_100_users['time'].dt.hour

In [45]:
# df_songs_heard_100_users.head()

In [46]:
#function to return slot number
def slot(hour):
    if 0 <= hour and hour <= 5:
        return 1
    elif 6 <= hour and hour <= 11:
        return 2
    elif 12 <= hour and hour <= 17:
        return 3
    else:
        return 4

In [47]:
# %%time
# df_songs_heard_100_users['slot'] = df_songs_heard_100_users['hour'].apply( lambda x : slot(x) ) 

In [48]:
# df_songs_heard_100_users.head()

In [49]:
# df_songs_heard_100_users.shape

Get count of songs and add to rest of the data. Use all slots.

In [50]:
# full dataser:
# user_songs_df.head()

In [51]:
%%time
user_song_df_count = user_songs_df.groupby(["userid","song"]).size().reset_index(name="count")

Wall time: 8.84 s


In [52]:
user_song_df_count.head()

,userid,song,count
0,user_000001,'84 Pontiac Dream,14
1,user_000001,(All I Wanted Was To Make You) Feel Good,3
2,user_000001,(D)Ecaying Beauty,1
3,user_000001,(Far From) Home,1
4,user_000001,(Get Off Your) High Horse Lady,1


convert into category datatype

In [53]:
%%time
# Convert userid and song names into numerical IDs
user_song_df_count['user_id'] = user_song_df_count['userid'].astype("category").cat.codes
user_song_df_count['song_id_new'] = user_song_df_count['song'].astype("category").cat.codes

# Create a lookup frame so we can get the artist names back in 
# readable form later.|
item_lookup = user_song_df_count[['song_id_new', 'song']].drop_duplicates()
item_lookup['song_id_new'] = item_lookup.song_id_new.astype(str)

Wall time: 5.54 s


In [54]:
item_lookup[:2]

,song_id_new,song
0,708,'84 Pontiac Dream
1,1456,(All I Wanted Was To Make You) Feel Good


In [55]:
item_lookup[item_lookup['song_id_new'] == str(702)]

,song_id_new,song
332777,702,'79 Aka The Shouty Track


In [56]:
user_song_df_count.head()

,userid,song,count,user_id,song_id_new
0,user_000001,'84 Pontiac Dream,14,0,708
1,user_000001,(All I Wanted Was To Make You) Feel Good,3,0,1456
2,user_000001,(D)Ecaying Beauty,1,0,1776
3,user_000001,(Far From) Home,1,0,2001
4,user_000001,(Get Off Your) High Horse Lady,1,0,2058


In [57]:
user_song_df_count.dtypes

userid         object
song           object
count           int64
user_id         int16
song_id_new     int32
dtype: object

In [58]:
user_song_df = user_song_df_count.drop( ['userid', 'song'], axis = 1)

In [59]:
user_song_df.head()

,count,user_id,song_id_new
0,14,0,708
1,3,0,1456
2,1,0,1776
3,1,0,2001
4,1,0,2058


#### Setting first row of every user_id as count=0

In [60]:
#user_song_df.loc[user_song_df[:100000].groupby('user_id')['count'].head(2).index, 'count']

In [61]:
# #user_song_df.loc[user_song_df.groupby('user_id')['count'].head(1).index, 'count'] = np.NaN
# # user_song_df.loc[user_song_df.groupby('user_id')['count'].head(1).index, 'count'] = 0

# M = 200000 #slice of df

# altered_songid_new_list = user_song_df.loc[user_song_df[:M].groupby('user_id')['song_id_new'].head(1).index, 'song_id_new']
# user_song_df.loc[user_song_df[:M].groupby('user_id')['count'].head(1).index, 'count'] = np.NaN
# altered_user_id_list = user_song_df.loc[user_song_df[:M].groupby('user_id')['user_id'].head(1).index, 'user_id']

In [62]:
#altered_user_id_list

In [63]:
# altered_songid_new_list

40 users changed

In [64]:
user_song_df.head()

,count,user_id,song_id_new
0,14,0,708
1,3,0,1456
2,1,0,1776
3,1,0,2001
4,1,0,2058


In [65]:
len(user_song_df) #before dropping: 4407910
#after: 4407870 

4407910

In [66]:
user_song_df = user_song_df.dropna()     #drop all rows that have any NaN values

user-item and count dataframe is ready

as count of songs played is an implicit feedback from users we will use ALS implicit model. 
Here, zeroes (absence) can signify 2 things: users do not like the song or have not yet heard that song.

Explicit feedback models only take dislike or like into account (like rating)

In [67]:
%%time
# Create lists of all users, songs and counts
users = list(np.sort(user_song_df.user_id.unique()))
songs = list(np.sort(user_song_df.song_id_new.unique()))
counts = list(user_song_df['count'])

Wall time: 335 ms


In [68]:
x = user_song_df[['user_id', 'song_id_new']]
y = user_song_df[['count']]

Creation of train and test data. ON HOLD

Cython imports for ALS

In [69]:
import sys
import pandas as pd
import numpy as np
import scipy.sparse as sparse
from scipy.sparse.linalg import spsolve
import random

from sklearn.preprocessing import MinMaxScaler

import implicit # The Cython library

In [70]:
# %%time
# #old one before altering DF
# sparse_item_user = sparse.csr_matrix((user_song_df_count['count'].astype(float), 
#                                       (user_song_df_count['song_id_new'], user_song_df['user_id'])))

# sparse_user_item = sparse.csr_matrix((user_song_df_count['count'].astype(float), 
#                                       (user_song_df_count['user_id'], user_song_df['song_id_new'])))

In [71]:
%%time
sparse_item_user = sparse.csr_matrix((user_song_df['count'].astype(float), 
                                      (user_song_df['song_id_new'], user_song_df['user_id'])))
##sparse_content_person = sparse_item_user
sparse_user_item = sparse.csr_matrix((user_song_df['count'].astype(float), 
                                      (user_song_df['user_id'], user_song_df['song_id_new'])))

Wall time: 380 ms


In [72]:
#sparsity
matrix_size = sparse_item_user.shape[0]*sparse_item_user.shape[1] # Number of possible interactions in the matrix
num_purchases = len(sparse_item_user.nonzero()[0]) # Number of items interacted with
sparsity = 100*(1 - (num_purchases/matrix_size))
sparsity

99.58988679334378

In [73]:
#after making first row of every user zero
sparse_item_user.shape
#old values: (1083471, 992)

(1083471, 992)

In [74]:
sparse_user_item.data

array([14.,  3.,  1., ...,  4.,  5.,  4.])

In [75]:
sparse_item_user.shape

(1083471, 992)

In [76]:
sparse_user_item.shape #transpose of above csr matrix

(992, 1083471)

In [77]:
%%time
# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

Wall time: 38.9 ms


We have the item-user data multiplied with the Confidence_Alpha_value.

In [78]:
%%time
# Fit the model
model.fit(data_conf)

100%|████████████████████████████████████████████████████████████████████████████████| 20.0/20 [00:12<00:00,  1.58it/s]


Wall time: 12.5 s


In [79]:


person_vecs = model.user_factors
content_vecs = model.item_factors

content_norms = np.sqrt((content_vecs * content_vecs).sum(axis=1))



In [157]:
person_vecs.shape

(992, 20)

In [186]:
content_vecs.shape

(1083471, 20)

In [191]:
person_vecs[1].dot(content_vecs[1].T).shape

()

In [206]:
person_vecs[:1][:].shape

(1, 20)

### CREATE USER RECOMMENDATIONS

In [80]:
## create function to recommend using ALS for user_id
def ALS_recommend_for_user(model, user_id, sparse_user_item, N = 10 ): #will add moe parameters later
    #user_id = 150

    # Use the implicit recommender.
    recommended = model.recommend(user_id, sparse_user_item, N)
    
    songs = []
    scores = []
    song_id_new_list = []
    
    # Get artist names from ids
    for item in recommended:
        idx, score = item
        songs.append(user_song_df_count.song.loc[user_song_df_count.song_id_new == idx].iloc[0])
        scores.append(score)
        song_id_new_list.append(idx)

    # Create a dataframe of artist names and scores
    recommendations = pd.DataFrame({'Song': songs, 'song_id_new': song_id_new_list, 'score': scores})

    print ("Top ", N, "recommendations for user_id:", user_id)
    return (recommendations)

##### This suggests that the recommendations given do not include previously existing items in the user's history

#### Creation of leave_one_out train and test dataset

In [81]:
import random

In [82]:
random.seed ( 10 ) #to set the seed and get the same randoms

In [145]:
def make_train(ratings, pct_test = 0.2):
    test_set = ratings.copy() # Make a copy of the original set to be the test set. 
    test_set[test_set != 0] = 1 # Store the test set as a binary preference matrix
    
    training_set = ratings.copy() # Make a copy of the original data we can alter as our training set. 
    
    nonzero_inds = training_set.nonzero() # Find the indices in the ratings data where an interaction exists
    nonzero_pairs = list(zip(nonzero_inds[0], nonzero_inds[1])) # Zip these pairs together of item,user index into list

    
    random.seed(0) # Set the random seed to zero for reproducibility
    
    num_samples = int(np.ceil(pct_test*len(nonzero_pairs))) # Round the number of samples needed to the nearest integer
    samples = random.sample(nonzero_pairs, num_samples) # Sample a random number of item-user pairs without replacement

    content_inds = [index[0] for index in samples] # Get the item row indices

    person_inds = [index[1] for index in samples] # Get the user column indices

    
    training_set[content_inds, person_inds] = 0 # Assign all of the randomly chosen user-item pairs to zero
    training_set.eliminate_zeros() # Get rid of zeros in sparse array storage after update to save space
    
    return training_set, test_set, list(set(person_inds))

#### Func to recommend

In [146]:
person_vecs[0,:].dot(content_vecs.T)

array([ 0.12057731,  0.05954903,  0.24297807, ..., -0.26500693,
        0.08844469,  0.05657415], dtype=float32)

In [148]:
def recommend(person_id, sparse_person_content, person_vecs, content_vecs, num_contents=10):
    # Get the interactions scores from the sparse person content matrix
    person_interactions = sparse_person_content[person_id,:].toarray()
    # Add 1 to everything, so that articles with no interaction yet become equal to 1
    person_interactions = person_interactions.reshape(-1) + 1
    # Make articles already interacted zero
    person_interactions[person_interactions > 1] = 0
    
    # Get dot product of person vector and all content vectors
    #rec_vector = person_vecs[person_id,:].dot(content_vecs.T).toarray() #got error as not array
    rec_vector = person_vecs[person_id,:].dot(content_vecs.T)
    
    # Scale this recommendation vector between 0 and 1
    min_max = MinMaxScaler()
    rec_vector_scaled = min_max.fit_transform(rec_vector.reshape(-1,1))[:,0]
    # Content already interacted have their recommendation multiplied by zero
    recommend_vector = person_interactions * rec_vector_scaled
    # Sort the indices of the content into order of best recommendations
    content_idx = np.argsort(recommend_vector)[::-1][:num_contents]
    
    # Start empty list to store titles and scores
    titles = []
    scores = []

    for idx in content_idx:
        # Append titles and scores to the list
        titles.append(user_song_df_count.song.loc[user_song_df_count.song_id_new == idx].iloc[0])
        scores.append(recommend_vector[idx])

    recommendations = pd.DataFrame({'title': titles, 'score': scores})

    return recommendations

### to alter csr matrix

In [149]:
# sparse_item_user
# sparse_user_item

In [150]:
user_id = 2

recommendations = recommend(user_id, sparse_user_item, person_vecs, content_vecs)

print(recommendations)

                                title     score
0                 Please Take Me Home  0.967708
1                                Mutt  0.967411
2                      Race The Dream  0.967175
3                        Online Songs  0.966329
4                     Izzo/In The End  0.963148
5                 Headfirst For Halos  0.957624
6                            Dumpweed  0.955220
7                  The Boogie Monster  0.950103
8                           In Stereo  0.947490
9  Love Rhymes With Hideous Car Wreck  0.942860


This returns a training set with masked values as '0'. Test set contains present or not present values for the items.

In [151]:
%%time
content_train, content_test, content_persons_altered = make_train(sparse_item_user, pct_test = 0.2)

Wall time: 5.61 s


In [152]:
def auc_score(predictions, test):
    fpr, tpr, thresholds = metrics.roc_curve(test, predictions)
    return metrics.auc(fpr, tpr)

In [225]:
def calc_mean_auc(training_set, altered_persons, predictions, test_set):
    store_auc = [] # An empty list to store the AUC for each user that had an item removed from the training set
    popularity_auc = [] # To store popular AUC scores
    pop_contents = np.array(test_set.sum(axis = 1)).reshape(-1) # Get sum of item iteractions to find most popular
    content_vecs = predictions[1]
    for person in altered_persons: # Iterate through each user that had an item altered
        training_column = training_set[:,person].toarray().reshape(-1) # Get the training set column
        zero_inds = np.where(training_column == 0) # Find where the interaction had not yet occurred
        
        # Get the predicted values based on our user/item vectors
        person_vec = predictions[0][person,:]
        # pred = person_vec.dot(content_vecs).toarray()[0,zero_inds].reshape(-1) #gave no toarray() error
        pred = person_vec.dot(content_vecs)[0,zero_inds].reshape(-1)
        
        # Get only the items that were originally zero
        # Select all ratings from the MF prediction for this user that originally had no iteraction
        actual = test_set[:,person].toarray()[zero_inds,0].reshape(-1)
        
        # Select the binarized yes/no interaction pairs from the original full data
        # that align with the same pairs in training 
        pop = pop_contents[zero_inds] # Get the item popularity for our chosen items
        
        store_auc.append(auc_score(pred, actual)) # Calculate AUC for the given user and store
        
        popularity_auc.append(auc_score(pop, actual)) # Calculate AUC using most popular and score
    # End users iteration
    
    return float('%.3f'%np.mean(store_auc)), float('%.3f'%np.mean(popularity_auc))

In [226]:
calc_mean_auc(content_train, content_persons_altered,
              [person_vecs, content_vecs.T], content_test)

IndexError: too many indices for array

In [161]:
pred = person_vec.dot(content_vecs)[0,zero_inds].reshape(-1)


ValueError: shapes (20,) and (1083471,20) not aligned: 20 (dim 0) != 1083471 (dim 0)

In [162]:
len(content_persons_altered)

987

In [212]:
[person_vecs, content_vecs.T][0][0,:]

array([ 5.830995  ,  3.4020638 ,  3.2037804 ,  0.06928024, -0.3612891 ,
        0.28846985,  0.47749293,  4.634209  , -1.5403954 ,  3.8320875 ,
        2.0402327 ,  2.1735811 ,  3.7980661 ,  3.461991  ,  2.286662  ,
        2.5447469 ,  2.6690295 ,  4.7803783 ,  5.904898  ,  1.26068   ],
      dtype=float32)

In [220]:
[person_vecs, content_vecs.T][0][0,:].shape

(20,)

In [165]:
#error: ValueError: shapes (20,) and (1083471,20) not aligned: 20 (dim 0) != 1083471 (dim 0)
person_vec = [person_vecs, content_vecs.T][0][content_persons_altered[0],:]

In [168]:
#worked
person_vec = [person_vecs, content_vecs.T][1][content_persons_altered[0],:]

In [169]:
person_vec.shape

(1083471,)

In [178]:
person_vec.dot(content_vecs)
#error 1 : ValueError: shapes (20,) and (1083471,20) not aligned: 20 (dim 0) != 1083471 (dim 0)

array([ 519.48047  ,   29.629044 ,  -63.04086  ,   34.786476 ,
       -139.58817  ,    9.586435 ,   -4.2970014,  -70.244965 ,
         15.411125 ,  -29.928411 ,  -29.859093 ,  -74.01828  ,
         -1.1909591,  -77.06438  ,  -74.80826  ,  -83.24075  ,
       -105.713585 ,  -26.002884 ,  -63.948456 ,  -78.89946  ],
      dtype=float32)

In [182]:
person_vec.dot(content_vecs).shape

(20,)

In [173]:
content_persons_altered[1]

1

In [129]:
# %%time
# #get train and tst set
# train_masked, test, users_altered, pairs_altered = make_train(sparse_user_item, pct_test = 0.00001)

In [130]:
# train_masked

In [131]:
# train_masked.data

In [132]:
# sparse_user_item #original train

In [133]:
# sparse_user_item.data #original train

In [134]:
#sparse_user_item.nonzero()

Now we have to train the model on the maked_training set and check whether the recommended items fall within the recommendations.

In [135]:
#pairs_altered[5]

In [136]:
# len(pairs_altered)

In [137]:
#sparse_user_item[ pairs_altered[55] ]

In [138]:
#train_masked[ pairs_altered[55] ]

a. Training the model

In [139]:
# %%time
# # Initialize the als model and fit it using the sparse item-user matrix
# model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

# # Calculate the confidence by multiplying it by our alpha value.
# alpha_val = 15
# data_conf = (train_masked * alpha_val).astype('double')

In [140]:
# %%time
# # Fit the model
# model.fit(data_conf)

In [141]:
# len(users_altered)

In [142]:
# users_altered[:3]

In [143]:
# recommendation_ans = (ALS_recommend_for_user(model, users_altered[0], train_masked.T, 20))
# print(recommendation_ans)

In [309]:
#pairs_altered

In [144]:
# sparse_user_item[ pairs_altered[0] ]

In [145]:
# pairs_altered[0][0]

In [146]:
# for i in pairs_altered:
#     print(pd.Series(recommendation_ans['Song'].isin(user_song_df_count [ user_song_df_count['user_id']== i[0] ] ['song']).values.astype(int), recommendation_ans['Song'].values))

In [147]:
# for i in pairs_altered:
#     print(pd.Series(recommendation_ans['Song'].isin(user_song_df_count [ user_song_df_count['user_id']== i[0] ] ['song']).values.astype(int), recommendation_ans['Song'].values)==1)

In [149]:
# altered_songid_new_list
# altered_user_id_list

#### recommend for altered users 
#### then check if the pred_songs exist in the actual_song_list

In [306]:
%%time
# Initialize the als model and fit it using the sparse item-user matrix
model = implicit.als.AlternatingLeastSquares(factors=20, regularization=0.1, iterations=20)

# Calculate the confidence by multiplying it by our alpha value.
alpha_val = 15
data_conf = (sparse_item_user * alpha_val).astype('double')

Wall time: 40.9 ms


In [307]:
%%time
# Fit the model
model.fit(data_conf)

100%|████████████████████████████████████████████████████████████████████████████████| 20.0/20 [00:19<00:00,  1.02it/s]


Wall time: 19.9 s


In [309]:
recommendation_ans = (ALS_recommend_for_user(model, altered_user_id_list[0], sparse_user_item, 50))
print (recommendation_ans)

Top  50 recommendations for user_id: 0
                                Song  song_id_new     score
0                                 Io       457060  1.346401
1                         Experience       296634  1.344260
2                          Transient       960093  1.333326
3                            January       468780  1.298728
4                             Embers       278893  1.292983
5                          I Can Fly       412538  1.291788
6                                111        17527  1.278692
7                           Pista 17       693713  1.271133
8                           Thoughts       940751  1.256536
9                            Keep On       486718  1.246562
10                         Battlecry       106559  1.229057
11                          Industry       449064  1.216999
12                         Self Help       776401  1.215383
13                        04 Faixa 4         9667  1.201816
14  Battle Scars (Feat. Willy Mason)       106489  1.201596
1

In [310]:
altered_songid_new_list[0]

708

In [311]:
# cnt_total = []
# for id in altered_user_id_list:
#     #print(pd.Series(recommendation_ans['Song'].isin(user_song_df_count [ user_song_df_count['user_id']== id ] ['song']).values.astype(int), recommendation_ans['Song'].values)==1)
#     ans = pd.Series(recommendation_ans['Song'].isin(user_song_df_count [ user_song_df_count['user_id']== id ] ['song']).values.astype(int), recommendation_ans['Song'].values)==1
#     cnt = 0
#     for i in ans:
#         if i == True:
#             cnt += 1 #increment if found true
#     cnt_total.append ( cnt ) #append to total found


In [312]:
type(altered_songid_new_list[0])

numpy.int32

In [313]:
altered_songid_new_list[0]

708

In [314]:
[item_lookup[ item_lookup['song_id_new'] == str(altered_songid_new_list[0])]['song']]

[0    '84 Pontiac Dream
 Name: song, dtype: object]

In [315]:
# %%time
# cnt_total = []
# index = 0
# for id in altered_user_id_list:
#     ans = pd.Series(recommendation_ans['Song'].isin([item_lookup[ item_lookup['song_id_new'] == str(altered_songid_new_list[0])]['song']]).values.astype(int), recommendation_ans['Song'].values)==1
#     cnt = 0
#     for i in ans:
#         if i == True:
#             cnt += 1 #increment if found true
#     cnt_total.append ( cnt ) #append to total found
#     index += 1

In [316]:
altered_songid_new_list[0]

708

In [317]:
%%time
cnt_total = []
index1 = 0
for id in altered_user_id_list:
    ans = pd.Series(recommendation_ans['song_id_new'].isin( [ altered_songid_new_list[index1] ] ), recommendation_ans['song_id_new'].values)==1
    cnt = 0
    for i in ans:
        if i == True:
            cnt += 1 #increment if found true
    cnt_total.append ( cnt ) #append to total found
    index += 1

Wall time: 34.9 ms


In [318]:
ans

457060     False
296634     False
960093     False
468780     False
278893     False
412538     False
17527      False
693713     False
940751     False
486718     False
106559     False
449064     False
776401     False
9667       False
106489     False
346041     False
573708     False
165439     False
564293     False
857905     False
595448     False
1048163    False
63158      False
543847     False
451776     False
287662     False
425250     False
832338     False
619845     False
601338     False
890851     False
260463     False
831758     False
701696     False
14431      False
134197     False
242714     False
276912     False
457194     False
695261     False
755346     False
905811     False
62758      False
1001152    False
708742     False
1000495    False
703419     False
744312     False
845022     False
371522     False
Name: song_id_new, dtype: bool

In [319]:
#cnt_total

recommender system precision, P = [# of our recommendations that are relevant] / [# of items we recommended]

P = relevant / N

In [320]:
N = len(recommendation_ans) #what N was set as, no. of items we recommended
precision = []

for i in range(len(cnt_total)): #loop 0 to N-1
    precision.append( cnt_total[i] / N )

In [321]:
cnt_total[5]

0

In [322]:
precision

[0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

### Generating training and testing data

In [58]:
#Group by users and get the size of each group
user_record_counts = df_songs_heard_100_users.groupby("userid").apply(lambda x: x.shape[0])

In [59]:
# for key, value in user_record_counts.items():
#     print (key)

In [60]:
%%time
#Divide dataset into train and test based on users and the number of the songs listened
train_set_keys = []
test_set_keys = []
sum_val = 0
total = df_songs_heard_100_users.shape[0]
for key, value in user_record_counts.items():
    sum_val += value
    fraction = sum_val/total
    if fraction <= 0.8:
        train_set_keys.append(key)        
    else:
        test_set_keys.append(key)

Wall time: 997 µs


In [61]:
print("Number of users in train set : "+str(len(train_set_keys)))
print("Number of users in test set : "+str(len(test_set_keys)))

Number of users in train set : 792
Number of users in test set : 198


In [62]:
df_songs_heard_100_users['userid'].iloc[81181]

'user_000018'

In [63]:
train_set = df_songs_heard_100_users.groupby("userid").filter(lambda x: x['userid'].iloc[0] in train_set_keys)
test_set = df_songs_heard_100_users.groupby("userid").filter(lambda x: x['userid'].iloc[0] in test_set_keys)

In [64]:
len(train_set)

3131754

In [65]:
len(test_set)

790347

In [66]:
print("Number of users in train set : "+str(train_set['userid'].nunique()))
print("Number of users in test set : "+str(test_set['userid'].nunique()))

Number of users in train set : 792
Number of users in test set : 198


In [67]:
print('Fraction of train set : ' + str(train_set.shape[0]/total))

Fraction of train set : 0.7984888711432979


In [68]:
train_set.head()

,userid,artistid,song,time,gender,age,country,hour,slot
22,user_000001,45bdb5be-ec03-484f-b58d-d22afc944b24,Clouds,2009-05-03 14:39:20,f,22.0,Germany,14,3
26,user_000001,ce559a88-58ba-4d8a-8456-9177412d609c,Seven,2009-05-03 14:07:40,f,22.0,Germany,14,3
28,user_000001,ce559a88-58ba-4d8a-8456-9177412d609c,Five,2009-05-03 13:56:25,f,22.0,Germany,13,3
30,user_000001,ce559a88-58ba-4d8a-8456-9177412d609c,Three,2009-05-03 13:40:46,f,22.0,Germany,13,3
39,user_000001,3d05eb8b-1644-4143-9a61-b28e33c4d85f,Something In The Way,2009-05-02 15:19:46,f,22.0,Germany,15,3


### Removing artistid, time and hour columns

In [69]:
train_set_df = train_set.drop(['artistid','time','hour'], axis=1, inplace=False)
test_set_df = test_set.drop(['artistid','time','hour'], axis=1, inplace=False)

In [70]:
train_set_df.head()

,userid,song,gender,age,country,slot
22,user_000001,Clouds,f,22.0,Germany,3
26,user_000001,Seven,f,22.0,Germany,3
28,user_000001,Five,f,22.0,Germany,3
30,user_000001,Three,f,22.0,Germany,3
39,user_000001,Something In The Way,f,22.0,Germany,3


### Creating train dataframes based on slots

In [71]:
grouped_train_set = train_set_df.groupby('slot')
grouped_test_set = test_set_df.groupby('slot')

In [72]:
train_first_slot_df = grouped_train_set.get_group(1)
train_second_slot_df = grouped_train_set.get_group(2)
train_third_slot_df = grouped_train_set.get_group(3)
train_fourth_slot_df = grouped_train_set.get_group(4)

test_first_slot_df = grouped_test_set.get_group(1)
test_second_slot_df = grouped_test_set.get_group(2)
test_third_slot_df = grouped_test_set.get_group(3)
test_fourth_slot_df = grouped_test_set.get_group(4)

In [73]:
train_first_slot_df.head()

,userid,song,gender,age,country,slot
124,user_000001,Carry On,f,22.0,Germany,1
138,user_000001,Carry On,f,22.0,Germany,1
361,user_000001,Intro,f,22.0,Germany,1
419,user_000001,Intro,f,22.0,Germany,1
971,user_000001,Sugar,f,22.0,Germany,1


In [74]:
print('First train slot shape : ' + str(train_first_slot_df.shape))
print('Second train slot shape : ' + str(train_second_slot_df.shape))
print('Third train slot shape : ' + str(train_third_slot_df.shape))
print('Fourth train slot shape : ' + str(train_fourth_slot_df.shape))
print('')
print("Number of users in train_first_slot_df : " + str(train_first_slot_df['userid'].nunique()))
print("Number of users in train_second_slot_df : " + str(train_second_slot_df['userid'].nunique()))
print("Number of users in train_third_slot_df : " + str(train_third_slot_df['userid'].nunique()))
print("Number of users in train_fourth_slot_df : " + str(train_fourth_slot_df['userid'].nunique()))
print('')
print("Number of songs in train_first_slot_df : " + str(train_first_slot_df['song'].nunique()))
print("Number of songs in train_second_slot_df : " + str(train_second_slot_df['song'].nunique()))
print("Number of songs in train_third_slot_df : " + str(train_third_slot_df['song'].nunique()))
print("Number of songs in train_fourth_slot_df : " + str(train_fourth_slot_df['song'].nunique()))
print('')
print('First test slot shape : ' + str(test_first_slot_df.shape))
print('Second test slot shape : ' + str(test_second_slot_df.shape))
print('Third test slot shape : ' + str(test_third_slot_df.shape))
print('Fourth test slot shape : ' + str(test_fourth_slot_df.shape))
print('')
print("Number of users in test_first_slot_df : " + str(test_first_slot_df['userid'].nunique()))
print("Number of users in test_second_slot_df : " + str(test_second_slot_df['userid'].nunique()))
print("Number of users in test_third_slot_df : " + str(test_third_slot_df['userid'].nunique()))
print("Number of users in test_fourth_slot_df : " + str(test_fourth_slot_df['userid'].nunique()))
print('')
print("Number of songs in test_first_slot_df : " + str(test_first_slot_df['song'].nunique()))
print("Number of songs in test_second_slot_df : " + str(test_second_slot_df['song'].nunique()))
print("Number of songs in test_third_slot_df : " + str(test_third_slot_df['song'].nunique()))
print("Number of songs in test_fourth_slot_df : " + str(test_fourth_slot_df['song'].nunique()))


First train slot shape : (596866, 6)
Second train slot shape : (544262, 6)
Third train slot shape : (943222, 6)
Fourth train slot shape : (1047404, 6)

Number of users in train_first_slot_df : 737
Number of users in train_second_slot_df : 762
Number of users in train_third_slot_df : 784
Number of users in train_fourth_slot_df : 780

Number of songs in train_first_slot_df : 4034
Number of songs in train_second_slot_df : 4034
Number of songs in train_third_slot_df : 4034
Number of songs in train_fourth_slot_df : 4034

First test slot shape : (173856, 6)
Second test slot shape : (146390, 6)
Third test slot shape : (221677, 6)
Fourth test slot shape : (248424, 6)

Number of users in test_first_slot_df : 185
Number of users in test_second_slot_df : 188
Number of users in test_third_slot_df : 191
Number of users in test_fourth_slot_df : 193

Number of songs in test_first_slot_df : 4034
Number of songs in test_second_slot_df : 4034
Number of songs in test_third_slot_df : 4034
Number of songs 

#### need to get count for each userid-song pair along with rest of the columns

In [138]:
# # Get user-song-count dataframe for each slot

# #train_user_song_count_df_first = train_first_slot_df.groupby(["userid","song"]).size().reset_index(name="count")
# #train1 = train_first_slot_df.groupby(["userid","song"]).size().reset_index(name="count")
# # test1 = 

# # df['Count'] = df.groupby(['Name'])['ID'].transform('count')
# # df.drop_duplicates()

# # train1 = train_first_slot_df.copy()
# # train1['count'] = train_first_slot_df.groupby(['userid','song'])['song'].transform('count')
# # train1 = train1.drop_duplicates()

# train_user_song_count_df_first = train_first_slot_df.copy()
# # train_user_song_count_df_first['Count'] = train_first_slot_df.groupby(['userid','song'])['song'].transform('count')
# train_user_song_count_df_first['count'] = train_first_slot_df.groupby(['userid','song'])['song'].transform('count')
# train_user_song_count_df_first = train_user_song_count_df_first.drop_duplicates()
# #train_user_song_count_df_first['count']= train_user_song_count_df_first['count'].astype(int)

# train_user_song_count_df_second = train_second_slot_df.groupby(["userid","song"]).size().reset_index(name="count")
# train_user_song_count_df_third = train_third_slot_df.groupby(["userid","song"]).size().reset_index(name="count")
# train_user_song_count_df_fourth = train_fourth_slot_df.groupby(["userid","song"]).size().reset_index(name="count")

# test_user_song_count_df_first = test_first_slot_df.groupby(["userid","song"]).size().reset_index(name="count")
# test_user_song_count_df_second = test_second_slot_df.groupby(["userid","song"]).size().reset_index(name="count")
# test_user_song_count_df_third = test_third_slot_df.groupby(["userid","song"]).size().reset_index(name="count")
# test_user_song_count_df_fourth = test_fourth_slot_df.groupby(["userid","song"]).size().reset_index(name="count")

In [144]:
# Get user-song-count dataframe for each slot

train_user_song_count_df_first = train_first_slot_df.copy()
train_user_song_count_df_first['count'] = train_first_slot_df.groupby(['userid','song'])['song'].transform('count')
train_user_song_count_df_first = train_user_song_count_df_first.drop_duplicates()

train_user_song_count_df_second = train_second_slot_df.copy()
train_user_song_count_df_second['count'] = train_second_slot_df.groupby(['userid','song'])['song'].transform('count')
train_user_song_count_df_second = train_user_song_count_df_second.drop_duplicates()


train_user_song_count_df_third = train_third_slot_df.copy()
train_user_song_count_df_third['count'] = train_third_slot_df.groupby(['userid','song'])['song'].transform('count')
train_user_song_count_df_third = train_user_song_count_df_third.drop_duplicates()

train_user_song_count_df_fourth = train_fourth_slot_df.copy()
train_user_song_count_df_fourth['count'] = train_fourth_slot_df.groupby(['userid','song'])['song'].transform('count')
train_user_song_count_df_fourth = train_user_song_count_df_fourth.drop_duplicates()


test_user_song_count_df_first = test_first_slot_df.copy()
test_user_song_count_df_first['count'] = test_first_slot_df.groupby(['userid','song'])['song'].transform('count')
test_user_song_count_df_first = test_user_song_count_df_first.drop_duplicates()

test_user_song_count_df_second = test_second_slot_df.copy()
test_user_song_count_df_second['count'] = test_second_slot_df.groupby(['userid','song'])['song'].transform('count')
test_user_song_count_df_second = test_user_song_count_df_second.drop_duplicates()

test_user_song_count_df_third = test_third_slot_df.copy()
test_user_song_count_df_third['count'] = test_third_slot_df.groupby(['userid','song'])['song'].transform('count')
test_user_song_count_df_third = test_user_song_count_df_third.drop_duplicates()

test_user_song_count_df_fourth = test_fourth_slot_df.copy()
test_user_song_count_df_fourth['count'] = test_fourth_slot_df.groupby(['userid','song'])['song'].transform('count')
test_user_song_count_df_fourth = test_user_song_count_df_fourth.drop_duplicates()


In [145]:
# del train1

In [146]:
#del train_user_song_count_df_first
len(train_user_song_count_df_first)

196612

In [147]:
train_user_song_count_df_first.head(10)
# len(train_user_song_count_df_first) : 196612

,userid,song,gender,age,country,slot,count
124,user_000001,Carry On,f,22.0,Germany,1,2
361,user_000001,Intro,f,22.0,Germany,1,2
971,user_000001,Sugar,f,22.0,Germany,1,3
975,user_000001,Summertime,f,22.0,Germany,1,3
1157,user_000001,Instrumental,f,22.0,Germany,1,1
1159,user_000001,Go Away,f,22.0,Germany,1,1
1163,user_000001,Earth Intruders,f,22.0,Germany,1,1
1164,user_000001,Porcelain,f,22.0,Germany,1,3
1220,user_000001,He Doesn'T Know Why,f,22.0,Germany,1,1
1224,user_000001,Blue Ridge Mountains,f,22.0,Germany,1,1


In [150]:
#train_user_song_count_df_first

In [151]:
test_user_song_count_df_first.head()

,userid,song,gender,age,country,slot,count
15511133,user_000802,Tomorrow,f,18.0,United States,1,2
15511365,user_000802,Tonight,f,18.0,United States,1,4
15511840,user_000802,Speechless,f,18.0,United States,1,11
15511850,user_000802,This Love,f,18.0,United States,1,10
15511852,user_000802,Secret,f,18.0,United States,1,11


In [152]:
# Get the max count for train and test set
train_max_count = test_user_song_count_df_first['count'].max()
test_max_count = test_user_song_count_df_first['count'].max()

## Surprise IGNORE TH

In [49]:
# Use the famous SVD algorithm.
algo = SVD(n_factors=250, n_epochs=250)
reader = surprise.reader.Reader(sep=',',rating_scale=(1, train_max_count))

In [50]:
train_data = surprise.Dataset.load_from_df(test_user_song_count_df_first[['userid', 'song', 'count']], reader)

In [51]:
#Use entire dataset as training data
trainset = train_data.build_full_trainset()

In [52]:
# Training
algo.fit(trainset)

In [57]:
test_data = test_user_song_count_df_first

In [59]:
# predict using test data
results = []
output = pd.DataFrame(columns = ['userid', 'song','score'])
for index, row in test_data.iterrows():
    prediction = algo.predict(row['userid'], row['song'], verbose=False)
#     key = str(str(row['userid']) + '-' + str(row['song']))
    predicted_rating = prediction.est
    output = output.append(pd.Series([str(row['userid']), str(row['song']), predicted_rating], index=output.columns), ignore_index=True)

KeyboardInterrupt: 

In [64]:
output[:1000]

,userid,song,score
0,user_000802,(Don'T Fear) The Reaper,431
1,user_000802,100%,431
2,user_000802,1979,431
3,user_000802,A Warm Place,431
4,user_000802,Addiction,431
...,...,...,...
995,user_000807,Decades,431
996,user_000807,Desperado,431
997,user_000807,Destiny,431
998,user_000807,Doctor Robert,431


In [63]:
test_data.shape

(51240, 3)